# 

# Hopefully you have python installed on your computer already, here's a list of things you need to do before any of this works!

## First you will need Jupyter Notebook so click the windows button and type "cmd", this will bring up the command prompt, open that and then copy this line here: -py m pip install jupyter notebook #

## Hopefully you are now reading this in Jupyter, we just need to run a few more pip installs; first start by opening a new window in the command prompt program using the plus icon, it should bring up windows powershell, that will work too!
## -py m pip install mplsoccer
## -py m pip install matplotlib
## -py m pip install plotly
## -py m pip install seaborn
## -py m pip install scipy
## -py m pip install pandas
## -py m pip install scikit-learn

# i think that's all of them but if it's not, just see what dependency is missing and run the line and then the module name!

# make sure you change the font to one you have installed, i'll include the gabarito fonts so that you can use them but you can use any font you wish

# make sure you change the database loading to wherever you have the db files installed, i'll upload the db files i am using right now with the program!

## i have commented throughout on the pizza chart function where to change various bits!

## to add new archetypes, literally just copy what i've done with the weights and stuff, if you don't want to weight, look at the pizza_plot_categories and put them in there, refer to them exactly as they are called in the spreadsheet, so Progressive Passes per 90 should be referred to as progPasses/90 etc. It's case sensitive.

## i've put everything in one big cell so you don't need to worry about running the code in order. I've tried to seperate the sections as best i can!

## radars and the dashboard function are probs both a bit fucked, radars less than the dashboard which just doesn't work!

## Some names are stored weirdly, Jhon Duran is Jader Duran, if you're not sure, look them up in the big spreadsheet.

In [10]:
# ##### import os
import os
import re
import base64
from io import BytesIO

import numpy as np
import pandas as pd
from scipy import stats

# Plotly
import plotly.graph_objects as go
import plotly.graph_objs as go  # You only need one, prefer 'graph_objects as go'
from plotly.colors import sample_colorscale, hex_to_rgb, sequential
from plotly.io import to_image
import plotly.io as pio

# Matplotlib & related
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import gridspec
from matplotlib import font_manager as fm
import matplotlib.colors as mcolors

# mplsoccer
from mplsoccer import PyPizza
from mplsoccer import Radar, grid
from matplotlib.patches import Circle

# Seaborn
import seaborn as sns

# Scikit-learn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# IPython/Jupyter widgets and display
from IPython.display import display, Javascript, HTML

import ipywidgets as widgets

### === Font Setup: Gabarito === ## change this to whatever font you want to use!
GABARITO_FONT_PATH = r"C:\\Users\\willi\\AppData\\Local\\Microsoft\\Windows\\Fonts\\Gabarito-Regular.ttf"
BOLD_FONT = r"C:\\Users\\willi\\AppData\\Local\\Microsoft\\Windows\\Fonts\\Gabarito-Bold.ttf"
font_normal = fm.FontProperties(fname=GABARITO_FONT_PATH)
font_bold = fm.FontProperties(fname=BOLD_FONT)

# ---- CATEGORY GROUPS FOR PIZZA CHART ----

position_groups = {
    'GK': ['GK'],
    'DF': ['CB', 'FB', 'WB', 'LB', 'RB', 'LWB', 'RWB', 'SW', 'DF'],
    'MF': ['DM', 'CM', 'AM', 'LM', 'RM', 'LW', 'RW', 'MF'],
    'FW': ['CF', 'ST', 'WF', 'FW']
}


pizza_plot_categories = {
    "Passing": ["compPass/90", "attPass/90", "pass%", "progPasses/90", "thirdPasses/90", "PPA/90", "xA/90", "kp/90", "xAG/90","tb/90"],
    "Defending": ["tackles/90", "Tkl+Int/90", "interceptions/90","pAdjtacklesWon/90","pAdjinterceptions/90", "clearances/90", "dribbledPast/90", "Blocked/90", "errors/90","shotsBlocked/90", "passesBlocked/90","tackleSuccessRate","ballRecoveries/90","midThirdTackles/90",
],
    "Carrying": ["progCarries/90", "thirdCarries/90", "Carries/90", "takeOnsAtt/90", "Succ/90","att3rdTouches/90","fouled/90"],
    "Shooting": ["goals/90", "Sh/90", "SoT/90", "npg/90","xG/90","SoT%","G/SoT","goals","xGOP/90","G/Sh"],
    "Aerial": ["headersWon/90","headersWon%"],
    "Ball Retention": [
    "touches/90",        # Completed passes per 90              # Pass completion %         # Passes attempted per 90         # Carries per 90
    "Dispossessed/90",    # Dispossessed per 90 (lower is better)
    "Mis/90",     # Miscontrols per 90 (lower is better)
    "sPass%",     # Short pass completion % (if you have it)      # Progressive passes
    "ballRecoveries/90",  # Ball recoveries per 90
],
}

category_palette = [
    "#2E4374", "#1A78CF", "#D70232", "#FF9300", "#44C3A1", "#CA228D", "#E1C340", "#7575A9", "#9DDFD3"
]
category_by_param = {}
for i, (cat, stats_list) in enumerate(pizza_plot_categories.items()):
    for stat in stats_list:
        category_by_param[stat] = category_palette[i % len(category_palette)]

greens_hex = [
    "#1fa44b",  # light mint
    "#1fa44b",
    "#1fa44b",
    "#1fa44b",
    "#1fa44b",
    "#1fa44b",
    "#158940",
    "#0e6d31",
    "#085222",
    "#053617",  # dark forest green
]
# ---- Display/column naming dictionaries ----
stat_display_names = {
    "tackles/90": "Tackles/90",
    "pAdjtacklesWon/90": "Possession Adjusted Tackles Won/90",
    "pAdjinterceptions/90" : " Possession Adjusted Interceptions/90",
    "Tkl+Int/90": "Tkl+Int/90",
    "interceptions/90": "Interceptions/90",
    "clearances/90": "Clearances/90",
    "dribbledPast/90": "Dribbled Past/90",
    "Blocked/90": "Blocks/90",
    "errors/90": "Errors/90",
    "Mis/90" : "Miscontrols/90",
    "passesBlocked/90": "Passes Blocked/90",
    "progCarries/90": "Progressive Carries/90",
    "thirdCarries/90": "Final Third Carries/90",
    "Carries/90": "Carries/90",
    "takeOnsAtt/90": "Take-Ons Attempted/90",
    "Succ/90": "Successful Dribbles/90",
    "compPass/90": "Passes Completed/90",
    "attPass/90": "Passes Attempted/90",
    "pass%": "Pass %",
    "progPasses/90": "Progressive Passes/90",
    "thirdPasses/90": "Passes to Final Third/90",
    "PPA/90": "Passes to Pen Area/90",
    "xA/90": "Expected Assists/90",
    "kp/90": "Key Passes/90",
    "goals/90": "Goals/90",
    "assists/90": "Assists/90",
    "Sh/90": "Shots/90",
    "SoT/90": "Shots on Target/90",
    "xG/90": "xG/90",
    "SCA90": "SCA/90",
    "GCA90": "GCA/90",
    "ballRecoveries/90": "Ball Recoveries/90",
    "headersWon%": "Headers Won %",
    "headersWon/90": "Headers Won/90",
    "tackleSuccessRate" : "Tackles Won %",
    "xGOP/90" : "xG Overperformance/90",
    "xAG/90" : "Expected Assisted Goals/90",
    "ga/90" : "Goal Contributions/90",
    "Dribblesucc%" : "Dribble Success %",
    "att3rdTouches/90" : "Attacking Third Touches/90",
    "shotsBlocked/90" : "Shots Blocked/90",
    "npg/90" : "Non Penalty Goals/90",
    "G/SoT" : "Goals/SoT",
    "SoT%" : "Shots On Target%",
    "goals" : "Goals",
    "assists" : "Assists",
    "tb/90" : "Through Balls/90",
    "midThirdTackles/90" : "Mid Third Tackles/90",
    "sPass%" : "Short Pass %",
    "G/Sh" : "Goals per Shot"
    
}

# Define the position weights first # change these as you want!
position_weights = {
    "CB - Ball-Playing": {
        "progPasses/90": 1.7,    # Most critical for ball-playing CBs
        "compPass/90": 1.6,
        "pass%": 1.9,
        "headersWon%": 1.4,       # Aerial ability still important
        "headersWon/90": 1.3,
        "interceptions/90": 1.3,
        "clearances/90": 1.2,
        "tackles/90": 1.2,
        "tackleSuccessRate": 1.8,
        "progCarryDist/90" : 1.5,
        "lPass%" : 1.5,
        "pAdjtacklesWon/90" : 1.1
    },
    "CB - Stopper": {
        "tackles/90": 1.8,       # Aggressive defending emphasized
        "Tkl+Int/90": 1.7,
        "clearances/90": 1.6,
        "headersWon/90": 1.6,
        "headersWon%": 1.5,
        "Blocked/90": 1.4,
        "interceptions/90": 1.3,
        "passesBlocked/90": 1.,
        "tackleSuccessRate": 1.95
    },
    "CB - Sweeper": {
        "interceptions/90": 1.7,  # Reading the game
        "clearances/90": 1.6,
        "tackles/90": 1.4,
        "progPasses/90": 1.4,     # Still needs some distribution
        "compPass/90": 1.3,
        "headersWon%": 1.2,
        "ballRecoveries/90": 1.5,
        "tackleSuccessRate": 1.7# Cleaning up behind defense
    },
    "FB/WB - Overlapping Attacker": {
        "crosses/90": 1.9,        # Key attacking output
        "compCross/90": 1.8,
        "tackleSuccessRate": 1.7, # Defensive reliability
        "assists/90": 1.6,
        "xAG/90": 1.5,
        "progCarries/90": 1.4,
        "PPA/90": 1.3,
        "thirdCarries/90": 1.2,
        "tackles/90" : 1.4
    },
    "FB/WB - Inverted": {
        "progPasses/90": 1.7,     # More playmaking focused
        "compPass/90": 1.6,
        "xAG/90": 1.5,
        "kp/90": 1.4,
        "progCarries/90": 1.4,
        "thirdCarries/90": 1.3,
        "tackleSuccessRate": 1.2,        # Some defensive contribution
        "interceptions/90": 1.1,
        "goals/90" : 1
    },

    "FB - Defensive": {
        "tackles/90": 1.6,
        "interceptions/90": 1.5,
        "clearances/90": 1.6,
        "passesBlocked/90": 1.5,
        "Blocked/90": 1.6,
        "defThirdTackles/90": 1.8,
        "ballRecoveries/90": 1.2,
        "sPass%": 1.1,
        "Dispossessed/90": 1.0,  # Lower is better, but still tracked
        "Mis/90": 1.0,
        "pass%": 1.0,
        "thirdPasses/90": 1.0,
        "crosses/90": 1.3,
        "dribbledPast/90": 0.6
},
    # ======= MIDFIELDERS ======= #
    "DM - Ball Winner": {
        "tackles/90": 1.9,        # Defensive dominance
        "Tkl+Int/90": 1.8,
        "interceptions/90": 1.7,
        "ballRecoveries/90": 1.6,
        "passesBlocked/90": 1.5,
        "clearances/90": 1.6,
        "progCarries/90": 1.2,    # Some ball progression
        "progPasses/90": 1.1,
        "shotsBlocked/90" : 1.3,
        "tackleSuccessRate" : 1.4,
        "defPenTouches/90" : 1.3,
        "defThirdTackles/90" : 1.5
    },
    "DM - Deep-Lying Playmaker": {
        "compPass/90": 1.9,     
        "progPasses/90": 1.8,
        "attPass/90": 1.9,
        "xA/90": 1.6,
        "thirdPasses/90": 1.5,
        "pass%": 1.7,
        "kp/90": 1.4,
        "tb/90" : 1.6,
        "interceptions/90": 1.2, 
        "tackleSuccessRate" : 1.2,
        "def3rdTouches/90" : 1.5,
        "progCarries/90" : 1.3
    },
    "CM - Box-to-Box Engine": {
        "Tkl+Int/90": 1.7,       # All-round contribution
        "progCarries/90": 1.7,
        "ballRecoveries/90": 1.6,
        "tackles/90": 1.6,
        "progPasses/90": 1.5,
        "thirdCarries/90": 1.4,
        "xA/90": 1.3,
        "xG/90": 1.2
    },
    "CM - Shuttler/Link Player": {
        "compPass/90": 1.7,       # Connective play
        "progPasses/90": 1.6,
        "Carries/90": 1.6,
        "thirdCarries/90": 1.2,
        "pass%": 1.5,
        "tackles/90": 1.5,
        "interceptions/90": 1.3,
        "ballRecoveries/90": 1.3,
        
        
    },
    "CM - Mezzala": {
        "progCarries/90": 1.8,    # Half-winger qualities
        "thirdCarries/90": 1.7,
        "xA/90": 1.6,
        "Sh/90": 1.3,
        "attPass/90": 1.4,
        "PPA/90": 1.3,
        "kp/90": 1.3,
        "tackles/90": 1.2, # Minimal defensive work
        "goals/90" : 1.1,
        "mid3rdTouches/90" : 1.7,
        "midThirdTackles/90" : 1.5,
    },
    "AM - Classic 10": {
        "xAG/90": 1.9,             # Creative hub
        "kp/90": 1.8,
        "SCA90": 1.7,
        "GCA90": 1.6,
        "PPA/90": 1.8,
        "assists/90": 1.5,
        "thirdPasses/90": 1.4,
        "progPasses/90": 1.2,
        "tb/90" : 1.3,
        "goals/90" : 1.2
    },
    # ======= FORWARDS ======= #
    "ST - Target Man": {
        "headersWon/90": 1.9,     # Aerial dominance
        "headersWon%": 1.9,   # Hold-up play
        "assists/90": 0.4,        # Knock-downs
        "xA/90": 1.2,
        "goals/90": 1.7,          # Secondary concern
        "npg/90": 1.8,
        "progPasses/90": 0.2
    },
    "ST - Poacher": {
        "xG/90": 1.35,             # Finishing quality
        "goals/90": 0.3,
        "npg/90": 1.6,
        "G/Sh": 1.7,              # Efficiency
        "SoT/90": 1,           # Overperformance
        "headersWon%": 1.4,
        "goals" : 1.9,
        "progCarries/90" : -0.4,
        "Distance" : 0.6# Less important (penalty box player)
    },
    "ST - Complete Forward": {
        "goals/90": 1.7,         # Balanced skillset
        "npg/90": 1.7,
        "assists/90": 1.6,
        "xA/90": 1.6,
        "headersWon/90": 1.5,
        "progCarries/90": 1.5,
        "Sh/90": 1.4,
        "kp/90": 1.4,
        "goals" : 1.2
    },
    "Winger - Classic": {
        "crosses/90": 1.9,        # Traditional winger
        "compCross/90": 1.8,
        "assists/90": 1.7,
        "xA/90": 1.7,
        "PPA/90": 1.6,
        "takeOnsAtt/90": 1.5,
        "progCarries/90": 1.4,
        "tackleSuccessRate": 1.2  # Tracking back
    },
    "Winger - Inverted": {
        "goals/90": 1.8,         # Goal threat
        "xG/90": 1.8,
        "Sh/90": 1.7,
        "progCarries/90": 1.6,
        "thirdCarries/90": 1.6,
        "xA/90": 1.5,
        "takeOnsAtt/90": 1.6,
        "SoT/90": 1.4,
        "kp/90" : 1.3
    },
    "AM - Shadow Striker": {
        "xG/90": 1.9,             # Secondary scorer
        "goals/90": 1.9,
        "npg/90": 1.8,
        "Sh/90": 1.7,
        "progCarries/90": 1.6,
        "SCA90": 1.5,
        "thirdCarries/90": 1.5,
        "SoT/90": 1.5,
        "xAG/90": 1.2
    }
}

# Role-specific score adjustments
position_adjustments = {
    "CB - Ball-Playing": lambda x: min(x * 1.03, 100),
    "CB - Stopper": lambda x: min(x * 1.05, 100),
    "FB/WB - Overlapping Attacker": lambda x: min(x * 1.04, 100),
    "DM - Ball Winner": lambda x: min(x * 1.06, 100),
    "DM - Deep-Lying Playmaker": lambda x: min(x * 1.07, 100),
    "ST - Target Man": lambda x: min(x * 1.05, 100),
    "Winger - Inverted": lambda x: min(x * 1.03, 100)
}



# Now define archetype_params using the same stats as position_weights
archetype_params = {}
for role, weights in position_weights.items():
    archetype_params[role] = list(weights.keys())

# ---- Add stat-category pizza options to the menu ----
category_archetypes = {f"{cat} Pizza": stats for cat, stats in pizza_plot_categories.items()}
archetype_params_full = {**archetype_params, **category_archetypes}
# -----------------------------------------------------
# ---- Set up the UI and load data ----

## CHANGE THIS TO WHATEVER FOLDER YOUR DB IS IN!!!!1
BASE = r"C:\Users\willi\Data Viz\FBREF scrapes"
pio.renderers.default = 'notebook'
FONT_FAMILY = "Gabarito"

# ==== MAIN MENU ============
print("\nChoose mode:")
print("1. Similar player comparison (bar chart + radar + clipboard)")
print("2. View percentiles for player (horizontal bar chart only)")
print("3. Pizza chart for player")
print("4. Top players by role (graphic leaderboard)")
print("5. Analyze player's best roles")
print("6. View Stat Leaders")
print("7. Generate radar plots for pizza categories")
print("8. Pizza Without The Toppings (For Dashboards)")
print("9. Lightmode Pizza Chart")
print("10. Lightmode Pizza Chart no toppings")
print("11. Ideal Partner Finder")
print("12. Scatter Plot generator")
print("13. Role Suitability Matrix")
mode = input("Enter 1-13: ").strip()
while mode not in ['1', '2', '3', '4', '5', '6', '7', '8','9','10',"11","12","13"]:
    print("Invalid selection. Please enter a number between 1-13.")
    mode = input("Enter 1-13: ").strip()

# ==== DATABASE SELECTION ====
print("\nWhich DB do you want to load?")
print("1. BigDB_ALL.csv (Minor Leagues)")
print("2. BigDB.csv (Big 5 European Leagues)")
print("3. Both")
mode_file = input("Enter 1, 2, or 3: ").strip()
files_to_load = []
if mode_file == "1":
    files_to_load.append(os.path.join(BASE, "BigDB_ALL.csv"))
elif mode_file == "2":
    files_to_load.append(os.path.join(BASE, "BigDB.csv"))
elif mode_file == "3":
    files_to_load.extend([
        os.path.join(BASE, "BigDB_ALL.csv"),
        os.path.join(BASE, "BigDB.csv"),
    ])
else:
    print("Invalid selection. Exiting.")
    raise SystemExit()

dfs = []
for p in files_to_load:
    if not os.path.isfile(p):
        print(f"❌ File not found: {p}")
        continue
    print(f"Loading: {p}")
    dfs.append(pd.read_csv(p))
if not dfs:
    raise ValueError("No valid files loaded.")
df = pd.concat(dfs, ignore_index=True)
print(f"Loaded {len(df)} total rows from {len(dfs)} file(s).")
df.head()

# === Position Filter - Handle comma-separated positions ===
print("\nAvailable positions: GK, DF, MF, FW")
while True:
    pos_input = input("Filter by position(s)? (comma sep, blank for all): ").strip().upper()
    if not pos_input:
        break
    selected_positions = [p.strip() for p in pos_input.split(",")]
    filtered_df = df.copy()
    mask = filtered_df['Pos'].apply(
        lambda x: any(pos in x.split(',') for pos in selected_positions) if isinstance(x, str) else False
    )
    filtered_df = filtered_df[mask]
    if len(filtered_df) == 0:
        print(f"No players found with positions: {', '.join(selected_positions)}")
    else:
        df = filtered_df
        print(f"Players after position filter: {len(df)}")
        break

# ---- Minutes played global filter (THIS IS THE ONLY PLACE YOU ASK) ----
min_minutes = input("\nEnter minimum minutes played (default 900): ").strip()
min_minutes = int(min_minutes) if min_minutes else 900

# --- Clean up mins before coercion ---
original_rows = len(df)
df['Mins'] = (
    df['Mins']
    .astype(str)
    .str.replace(',', '', regex=False)
)
print(f"Rows after string cleanup: {len(df)}")

df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
print(f"Rows after numeric coercion: {len(df)}")

df = df[df['Mins'] >= min_minutes]
print(f"Rows with {min_minutes}+ mins: {len(df)}")
print(f"Final rows after filter: {len(df)}")

# ---- Utility to use this filter in all downstream stats functions, just in case ----
def filter_by_minutes(df, min_minutes):
    df = df.copy()
    df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
    return df[df['Mins'] >= min_minutes]

# --- Player selection (skip if mode is 5) ---
player_row = None
stat_cols = None
stat_labels = None
arch_idx = None
if mode not in ["4", "6", "12", "13"]:
    # Only ask for player selection if not in mode 4 or 6
    player_name = input("\nEnter player name (case-insensitive): ").strip().lower()
    matches = df[df['Player'].str.lower() == player_name]
    if matches.empty:
        print("No player found!")
        raise SystemExit()
    elif len(matches) > 1:
        print("Multiple matches:")
        print(matches[["Player", "Squad", "Pos", "Age", "Mins"]])
        idx = int(input(f"Choose a row (0 to {len(matches)-1}): "))
        player_row = matches.iloc[idx]
    else:
        player_row = matches.iloc[0]

    # Only prompt for archetype if NOT dashboard mode (mode 6) or role analyzer (mode 5)
    if mode not in ["5", "6", "11", "13"]:
        archetype_keys = list(archetype_params_full.keys())
        print("\nArchetypes:")
        for i, k in enumerate(archetype_keys, 1):
            print(f"{i:2d}. {k}")

        while True:
            try:
                arch_idx = int(input("\nChoose archetype by number: ")) - 1
                if 0 <= arch_idx < len(archetype_keys):
                    break
                print(f"Please enter a number between 1 and {len(archetype_keys)}")
            except ValueError:
                print("Please enter a valid number")

        stat_cols = archetype_params_full[archetype_keys[arch_idx]]
        stat_labels = [stat_display_names.get(stat, stat) for stat in stat_cols]

# ---- UTILS ----
def position_relative_percentile(df, player_row, stat_col):
    """Return the player's percentile for stat_col vs only players with any of their positions."""
    pos_str = player_row.get('Pos', '')
    positions = [p.strip() for p in str(pos_str).split(',') if p.strip()]
    if not positions or stat_col not in df.columns:
        return np.nan
    mask = df['Pos'].apply(
        lambda s: any(pos in str(s).split(',') for pos in positions) if isinstance(s, str) else False
    )
    position_df = df[mask]
    stat_vals = position_df[stat_col].replace([np.inf, -np.inf], np.nan).dropna()
    player_val = player_row.get(stat_col, np.nan)
    if pd.isnull(player_val) or stat_vals.empty:
        return np.nan
    return round(stats.percentileofscore(stat_vals, player_val, kind='mean'), 2)


def safe_int(val):
    try:
        return int(str(val).replace(',', '').replace(" ", "").strip())
    except Exception:
        return 0


# First, let's update the calculate_role_score function to be more consistent
def calculate_role_score(player_row, role_stats, df, role_name=None):
    if not role_stats:
        return 50
    df = filter_by_minutes(df, min_minutes)   # <-- always apply min_minutes
    pos_str = player_row.get('Pos', '')
    player_positions = [p.strip() for p in str(pos_str).split(',')] if isinstance(pos_str, str) else []
    pos_filtered_db = df[df['Pos'].apply(
        lambda s: any(pos in str(s).split(',') for pos in player_positions) if isinstance(s, str) else False
    )] if player_positions else df

    percentiles = []
    weights = []
    for stat in role_stats:
        stat_vals = pos_filtered_db[stat].replace([np.inf, -np.inf], np.nan).dropna()
        player_val = player_row.get(stat, np.nan)
        if pd.notnull(player_val) and not stat_vals.empty:
            pctl = stats.percentileofscore(stat_vals, player_val, kind='mean')
            weight = position_weights.get(role_name, {}).get(stat, 1.0)
            percentiles.append(pctl)
            weights.append(weight)
    if not percentiles:
        return 50
    if len(percentiles) >= 3:
        role_score = np.average(percentiles, weights=weights)
    else:
        role_score = np.mean(percentiles)
    adjustment_func = position_adjustments.get(role_name, lambda x: x)
    role_score = adjustment_func(role_score)
    return min(100, max(0, round(role_score, 1)))

def show_player_summary(player_row, stat_cols, stat_display_names, df):
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)  # Fallback to stat_cols if not in archetype_params
    
    # Calculate role score using the consistent method
    role_score = calculate_role_score(player_row, role_stats, df, role_name)
    
    percentiles = []
    for stat in stat_cols:
        p = position_relative_percentile(df, player_row, stat)
        percentiles.append(round(p, 2) if not np.isnan(p) else np.nan)
    
    lines = [
        f"{player_row['Player']} (Age: {player_row.get('Age', '?')}, Club: {player_row.get('Squad', '?')}, Minutes: {player_row.get('Mins', '?')})",
        f"Role: {role_name} | Role Score: {role_score:.1f}/100"
    ]
    
    for stat, p in zip(stat_cols, percentiles):
        val = player_row.get(stat, np.nan)
        if pd.notnull(val):
            lines.append(f"{stat_display_names.get(stat, stat)}: {val:.2f} (Percentile: {p:.2f}%)")
        else:
            lines.append(f"{stat_display_names.get(stat, stat)}: N/A")
    
    lines.append("\nData generated by @nstntly")
    summary_text = "\n".join(lines)
    
    # Rest of the function remains the same...
    print("\n" + "="*55)
    print(summary_text)
    print("="*55 + "\n")
    
    textarea = widgets.Textarea(
        value=summary_text,
        description='',
        layout=widgets.Layout(width='420px', height='320px')
    )
    button = widgets.Button(description="Copy to clipboard", button_style='success')
    js = f"""navigator.clipboard.writeText(`{summary_text}`);"""
    
    def on_button_click(b):
        try:
            display(Javascript(js))
            print("Copied to clipboard!")
        except Exception:
            print("Copy manually from the box above.")
    
    button.on_click(on_button_click)
    display(widgets.HBox([textarea, button]))

def show_percentile_bar_chart():
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    
    # Calculate role score using consistent method
    role_score = calculate_role_score(player_row, role_stats, df, role_name)
    
    vals = player_row[stat_cols].values
    percentiles = [
        position_relative_percentile(df, player_row, stat) if np.isfinite(v) else 0
        for stat, v in zip(stat_cols, vals)
    ]
    
    # Create the bar trace
    bar = go.Bar(
        x=percentiles,
        y=[stat_display_names.get(stat, stat) for stat in stat_cols],
        orientation='h',
        text=[f"{p:.2f}%" for p in percentiles],
        textposition="auto",
        marker=dict(color=percentiles, colorscale="RdYlGn"),
    )

    # Update title to include role score
    layout = go.Layout(
        title=dict(
            text=f"{player_row['Player']}<br><sup>{archetype_keys[arch_idx]} | Role Score: {role_score:.1f}/100</sup>",
            x=0.5,
            font=dict(family=FONT_FAMILY, size=22, color="#111")
        ),
        font=dict(family=FONT_FAMILY, size=14, color="#222"),
        xaxis=dict(title="Percentile", range=[0, 100], showgrid=True, gridcolor="#eee"),
        yaxis=dict(automargin=True, tickfont=dict(size=15)),
        margin=dict(l=180, r=40, t=80, b=40),
        template="simple_white",
        height=50+35*len(stat_cols),
        width=700,
        plot_bgcolor="white",
        paper_bgcolor="white",
    )
    fig = go.FigureWidget([bar], layout=layout)
    display(fig)


def show_pizza_chart(player_row, stat_cols, stat_labels, df, min_minutes):
    # Coerce Mins to numeric in the DataFrame
    df_filtered = df.copy()
    df_filtered['Mins'] = pd.to_numeric(df_filtered['Mins'], errors='coerce')
    df_filtered = df_filtered[df_filtered['Mins'] >= min_minutes]

    # Coerce Mins for the player row (handles cases where mins is string or NaN)
    try:
        player_mins = player_row.get('Mins', 0)
        player_mins = pd.to_numeric(player_mins, errors='coerce')
    except Exception:
        player_mins = 0

    if pd.isna(player_mins) or player_mins < min_minutes:
        print(f"Player {player_row.get('Player','?')} does not meet the minimum minutes requirement ({player_mins} < {min_minutes}).")
        return

    # All role/percentile calculations are now done with filtered dataframe!
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    raw_stat_values = [player_row.get(s, float('nan')) for s in stat_cols]

    # Calculate role score using filtered DataFrame
    role_score = calculate_role_score(player_row, role_stats, df_filtered, role_name)

    # Calculate percentiles for each stat (filtered df)
    percentiles = [
        position_relative_percentile(df_filtered, player_row, stat) if np.isfinite(player_row.get(stat, np.nan)) else 0
        for stat in stat_cols
    ]

    # --- DYNAMIC CONTRAST COLOUR LOGIC ---
    def get_contrast_text_color(hex_color):
        r, g, b = mcolors.hex2color(hex_color)
        brightness = (r*299 + g*587 + b*114) * 255 / 1000
        return "#000000" if brightness > 140 else "#F2F2F2"

    slice_colors = [category_by_param.get(s, "#2E4374") for s in stat_cols]
    text_colors = [get_contrast_text_color(c) for c in slice_colors]

    def break_label(label, max_len=15):
        if label.endswith("/90"):
            before = label[:-3].rstrip()
            if len(label) <= max_len:
                return label
            words = before.split(" ")
            lines = []
            current_line = ""
            for word in words:
                check_len = len(current_line + (" " if current_line else "") + word + "/90")
                if len(lines) == 0 and check_len > max_len:
                    if current_line:
                        lines.append(current_line)
                    current_line = word
                elif len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            lines.append(current_line + "/90")
            return "\n".join(lines)
        else:
            if len(label) <= max_len:
                return label
            words = label.split(" ")
            lines = []
            current_line = ""
            for word in words:
                if len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            if current_line:
                lines.append(current_line)
            return "\n".join(lines)

    display_params = [break_label(stat_display_names.get(p, p), max_len=15) for p in stat_cols]

    figsize = (10, 11)
    baker = PyPizza(
        params=display_params,
        background_color="#222222",
        straight_line_color="#000000",
        straight_line_lw=.3,
        last_circle_color="#000000",
        last_circle_lw=1,
        other_circle_lw=0,
        inner_circle_size=0.3
    )
    fig, ax = baker.make_pizza(
        percentiles,
        alt_text_values=raw_stat_values,
        figsize=figsize,
        color_blank_space="same",
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(
            edgecolor="#000000", zorder=2, linewidth=1
        ),
        kwargs_params=dict(
            color="#fffff0", fontsize=18,
            fontproperties=font_normal, va="center"
        ),
        kwargs_values=dict(
            color="#fffff0", fontsize=15,
            fontproperties=font_normal, zorder=3,
            bbox=dict(
                edgecolor="#000000", facecolor="cornflowerblue",
                boxstyle="round,pad=0.2", lw=1
            )
        )
    )

    club = player_row.get("Squad", "?")
    nationality = player_row.get("Nation", "?")
    age = player_row.get("Age", "?")
    mins_played = player_row.get("Mins", 0)
    season = "2024/25"
    chosen_role = archetype_keys[arch_idx]
    if isinstance(selected_positions, list):
        positions_label = ", ".join(str(pos) for pos in selected_positions)
    else:
        positions_label = str(selected_positions)

    # Player name and heading
    fig.text(
        0.5, 0.985, player_row['Player'], ha='center', va='top', fontsize=22,
        fontweight='bold', color='#fffff0', fontproperties=font_normal
    )
    fig.text(
        0.5, 0.952, f"{club} | {nationality} | {age} | {season}", ha='center', va='top', fontsize=15,
        color='#fffff0', fontproperties=font_normal
    )
    fig.text(
        0.5, 0.928, f"Compared vs: {positions_label} | Role: {chosen_role} | Minutes played: {mins_played}",
        ha='center', va='top', fontsize=12, color='#fffff0', fontproperties=font_normal
    )
    # ---- Add this line just before the social handle
    fig.text(
        0.5, 0.08, f"Filtered by minimum minutes: {min_minutes}",
        ha='center', va='top', fontsize=11, color='#CCCCCC',
        fontproperties=font_normal, alpha=0.88
    )
    # Social handle/credit
    fig.text(
        0.5, 0.01, "Created by @nstntly | Inspired by @booteful_game",
        ha='center', va='bottom', fontsize=9, color='#CCCCCC', fontproperties=font_normal, alpha=0.8
    )
    # role score attachment
    fig.text(
        0.845, 0.9, f"Role Score: {role_score:.1f}",
        ha="center", va="top",
        fontsize=14,
        fontproperties=font_normal,
        fontweight="bold",
        color="#f1ffcd",
        bbox=dict(
            boxstyle="round,pad=0.1",
            facecolor="#1A78CF" if role_score >= 70 else "#D70232" if role_score < 40 else "#FF9300",
            edgecolor="#000000",
            linewidth=1
        ),
        zorder=5
    )
    # save logic, saves to whatever folder the program is in!
    save_name = f"pizza_{player_row['Player'].replace(' ', '_')}_{season.replace('/', '-')}.png"
    plt.savefig(save_name, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.show()
    print(f"Pizza plot saved as {save_name}")

######################################################################################################################################################
######################################################################################################################################################
######################################################################################################################################################
######################################################################################################################################################
######################################################################################################################################################
def show_similarity_comparison(player_row, stat_cols, stat_labels, df, min_minutes):
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    
    # --- Apply min_minutes filter for ALL calculations ---
    df_filtered = df.copy()
    df_filtered['Mins'] = pd.to_numeric(df_filtered['Mins'], errors='coerce')
    df_filtered = df_filtered[df_filtered['Mins'] >= min_minutes]
    
    # Calculate base role score using consistent method
    base_role_score = calculate_role_score(player_row, role_stats, df_filtered, role_name)
    
    # Calculate similarities
    X = df_filtered[stat_cols].fillna(0)
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    player_vec = scaler.transform([player_row[stat_cols].fillna(0)])
    similarities = cosine_similarity(player_vec, X_scaled)[0]
    
    # Get top similar players (excluding the player themselves if present)
    df_filtered = df_filtered.reset_index(drop=True)
    try:
        player_idx_in_filtered = df_filtered.index[df_filtered['Player'] == player_row['Player']][0]
        similarities[player_idx_in_filtered] = -1  # Ensure player doesn't match with themselves
    except Exception:
        pass
    
    sim_order = np.argsort(-similarities)
    top_indices = sim_order[:10]
    top_players = df_filtered.iloc[top_indices]
    top_sims = similarities[top_indices]
    
    # Calculate role scores for similar players (filtered by minutes!)
    top_role_scores = []
    for _, row in top_players.iterrows():
        score = calculate_role_score(row, role_stats, df_filtered, role_name)
        top_role_scores.append(score)
    
    # Prepare hover text
    hovertexts = []
    for i, (idx, row) in enumerate(top_players.iterrows()):
        sim_score = round(top_sims[list(top_players.index).index(idx)] * 100, 2)
        role_score = top_role_scores[i]
        lines = [
            f"<b>{row['Player']}</b> <span style='color:#888;font-size:13px;'>({row['Squad']}, {row.get('Nation','?')}, Age {row.get('Age','?')})</span><br>",
            f"Similarity: <b>{sim_score:.2f}%</b> &nbsp;|&nbsp; <span style='color:#1A78CF'>Role Score: {role_score:.1f}</span>"
        ]
        for stat in stat_cols:
            user_val = row[stat]
            base_val = player_row[stat]
            p = position_relative_percentile(df_filtered, row, stat)
            lines.append(
                f"{stat_display_names.get(stat, stat)}: <b>{user_val:.2f}</b> <span style='color:#666'>(Baseline {player_row['Player']}: {base_val:.2f}, Pos. Percentile: {p:.2f}%)</span>"
            )
        hovertexts.append("<br>".join(lines))

    top_players = top_players.iloc[::-1]
    top_sims = top_sims[::-1]
    top_role_scores = top_role_scores[::-1]
    hovertexts = hovertexts[::-1]

    # Create the bar chart
    bar_trace = go.Bar(
        y=top_players['Player'],
        x=top_sims * 100,
        orientation='h',
        text=[f"{s*100:.2f}%" for s in top_sims],
        textposition='inside',
        insidetextanchor='middle',
        marker=dict(
            color=greens_hex,
            line=dict(width=0)
        ),
        hovertext=hovertexts,
        hoverinfo='text',
    )
    
    fig = go.FigureWidget([bar_trace])
    fig.update_layout(
        title=dict(
            text=f"10 Most similar players to {player_row['Player']}<br><sup>Archetype: <b>{role_name}</b> | Baseline Role Score: {base_role_score:.1f}</sup>",
            x=0.5,
            font=dict(size=64, family=FONT_FAMILY, color="#111"),
        ),
        plot_bgcolor="#f1ffcd",
        paper_bgcolor="#f1ffcd",
        font=dict(family=FONT_FAMILY, size=42, color="#222"),
        xaxis=dict(
            title="Similarity (%)",
            gridcolor="#f1ffcd",
            range=[0, 100]
        ),
        yaxis=dict(
            title="",
            automargin=True,
            tickfont=dict(size=45),
        ),
        margin=dict(l=180, r=40, t=90, b=40),
        hoverlabel=dict(bgcolor="white", font_size=13, font_family=FONT_FAMILY),
        height=1800, width=2700,
        template="simple_white"
    )
    
    # Set up interactive components
    radar_out = widgets.Output()
    clipboard_out = widgets.Output()

    def plot_radar(player1_row, player2_row, stat_cols, player2_name, db, pretty_labels, archetype_label):
        radar_vals = []
        base_vals = []
        for stat in stat_cols:
            radar_vals.append(position_relative_percentile(db, player2_row, stat))
            base_vals.append(position_relative_percentile(db, player1_row, stat))
        stat_labels = [pretty_labels.get(stat, stat) for stat in stat_cols]
        radar = go.Figure()
        radar.add_trace(go.Scatterpolar(
            r=base_vals,
            theta=stat_labels,
            fill='toself',
            name=f"{player1_row['Player']} (Percentile)",
            line_color='green'
        ))
        radar.add_trace(go.Scatterpolar(
            r=radar_vals,
            theta=stat_labels,
            fill='toself',
            name=f"{player2_name} (Percentile)",
            line_color='blue'
        ))
        radar.update_layout(
            polar=dict(
                radialaxis=dict(visible=True, range=[0, 100], showgrid=True)
            ),
            showlegend=True,
            height=500,
            width=600,
            font=dict(family=FONT_FAMILY),
            title=f"<b>{player1_row['Player']} vs {player2_name}</b> — {archetype_label}"
        )
        return radar

    def on_bar_click(trace, points, selector):
        clipboard_out.clear_output()
        radar_out.clear_output()
        if not points.point_inds:
            return
        i = points.point_inds[0]
        row = top_players.iloc[i]
        role_score = top_role_scores[i]
        lines = [
            f"{row['Player']} (Age: {row.get('Age', '?')}, Club: {row.get('Squad', '?')}, Minutes: {row.get('Mins', '?')})",
            f"Role Score: {role_score:.1f}/100 (for role: {role_name})"
        ]
        for stat in stat_cols:
            val = row[stat]
            p = position_relative_percentile(df_filtered, row, stat)
            lines.append(f"{stat_display_names.get(stat, stat)}: {val:.2f} (Percentile: {p:.2f}%)")
        lines.append(f"\nBaseline: {player_row['Player']} role score = {base_role_score:.1f}/100")
        lines.append("\nData generated by @nstntly")
        stats_to_copy = "\n".join(lines)
        with clipboard_out:
            textarea = widgets.Textarea(
                value=stats_to_copy,
                description='',
                layout=widgets.Layout(width='420px', height='220px')
            )
            button = widgets.Button(description="Copy to clipboard", button_style='success')
            js = f"""navigator.clipboard.writeText(`{stats_to_copy}`);"""
            def on_button_click(b):
                try:
                    display(Javascript(js))
                    print("Copied to clipboard!")
                except:
                    print("Copy manually from the box above.")
            button.on_click(on_button_click)
            display(widgets.HBox([textarea, button]))
        with radar_out:
            radar_fig = plot_radar(
                player_row, row, stat_cols, row['Player'],
                db=df_filtered,
                pretty_labels=stat_display_names,
                archetype_label=role_name
            )
            radar_fig.show()
    
    fig.data[0].on_click(on_bar_click)
    display(fig)
    display(widgets.HTML("<br><b>Click a player to compare on radar plot and show stats (with copy button below).</b>"))
    display(radar_out)
    display(clipboard_out)



    
    
    
##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################
##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################    
def show_role_leaderboard(df, position_weights, stat_display_names, min_minutes):
    from IPython.display import display as ipy_display, Javascript
    import ipywidgets as widgets
    import warnings
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    import matplotlib.colors as mcolors
    import plotly.graph_objs as go
    FONT_FAMILY = "Gabarito, Montserrat, Arial, sans-serif"

    # --- LEAGUE FILTER PROMPT ---
    # Extract unique league names, stripping country prefix (e.g. 'eng Premier League' -> 'Premier League')
    # --- LEAGUE PREFIX STRIPPER ---
    def strip_main_prefix(comp):
        prefixes = ('eng ', 'it ', 'es ', 'de ', 'fr ')
        for prefix in prefixes:
            if comp.lower().startswith(prefix):
                return comp[len(prefix):]
        return comp

    print("\nAvailable roles:")
    role_keys = list(position_weights.keys())
    for i, role in enumerate(role_keys, 1):
        print(f"{i:2d}. {role}")

    role_idx = int(input("\nEnter role number: ")) - 1
    if role_idx < 0 or role_idx >= len(role_keys):
        print("Invalid role selection")
        return

    max_age = input("Enter maximum age (leave blank for no limit): ").strip()
    max_age = int(max_age) if max_age else None

    # --- Minutes filter ---
    df = df.copy()
    df['Mins'] = pd.to_numeric(df['Mins'].astype(str).str.replace(',', ''), errors='coerce')
    filtered_df = df[df['Mins'] >= min_minutes]
    if max_age:
        filtered_df = filtered_df[filtered_df['Age'] <= max_age]

    role_name = role_keys[role_idx]
    role_stats_dict = position_weights[role_name]
    role_stats = list(role_stats_dict.keys())

    # --- Calculate role scores for all players using the min-minutes (but NOT yet filtering by league) ---
    print("\nCalculating role scores for all players...")
    filtered_df = filtered_df.copy()
    filtered_df['RoleScore'] = [
        calculate_role_score(row, role_stats, filtered_df, role_name)
        for _, row in filtered_df.iterrows()
    ]

    # --- Add LeagueName column to make league filtering easier ---
    filtered_df['LeagueName'] = filtered_df['Comp'].astype(str).apply(strip_main_prefix)
    leagues = filtered_df['LeagueName'].unique()
    print("\nAvailable leagues loaded:")
    print(", ".join(leagues))
    league_input = input("Filter by league(s)? (comma sep, blank for all): ").strip()
    if league_input:
        chosen_leagues = [l.strip().lower() for l in league_input.split(",") if l.strip()]
        mask = filtered_df['LeagueName'].str.lower().isin(chosen_leagues)
        if not mask.any():
            print("No matches found for input! Please check league names as shown above.")
            return
        filtered_df = filtered_df[mask]
        print(f"Players after league filter: {len(filtered_df)}")
    else:
        print("No league filter applied.")

    # Now you can drop LeagueName if you want
    filtered_df = filtered_df.drop(columns=["LeagueName"])
    filtered_df = filtered_df.copy()
    filtered_df['RoleScore'] = [
    calculate_role_score(row, role_stats, filtered_df, role_name)
    for _, row in filtered_df.iterrows()
]
    top_players = filtered_df.nlargest(10, 'RoleScore').reset_index(drop=True)

    # --- Gradient color setup (light to dark blue) ---
    blue_gradient = [
        "#B2DFFC", "#8EC3E0", "#6FA7D6", "#578BC8", "#4370BA",
        "#3457A4", "#274097", "#1F2C7B", "#1A3474", "#112052"
    ][::-1]  # dark at top, light at bottom

    bar_colors = blue_gradient[-len(top_players):][::-1] if len(top_players) <= 10 else \
        [blue_gradient[0] for _ in range(len(top_players))]

    def luminance(hexcolor):
        rgb = mcolors.hex2color(hexcolor)
        r, g, b = [x * 255 for x in rgb]
        return 0.299*r + 0.587*g + 0.114*b

    bar_text_colors = ['white' if luminance(c) < 150 else '#222' for c in bar_colors]

    bar_labels = []
    for _, row in top_players.iterrows():
        bar_labels.append(
            f"{row['Player']} • {row['RoleScore']:.1f} • {row.get('Age', '?')} • {row.get('Squad', '?')} • {int(row.get('Mins',0)):,} mins"
        )

    fig = go.FigureWidget([
        go.Bar(
            x=top_players['RoleScore'],
            y=[f"#{i+1}" for i in range(len(top_players))],
            orientation='h',
            text=bar_labels,
            textposition='inside',
            insidetextanchor='middle',
            marker=dict(
                color=bar_colors,
                line=dict(color='#333333', width=1)
            ),
            textfont=dict(
                color=bar_text_colors,
                size=13,
                family=FONT_FAMILY
            ),
            customdata=top_players.index.values,
            hovertext=[f"{row['Player']} ({row['Squad']})" for _, row in top_players.iterrows()],
            hoverinfo="text"
        )
    ])

    fig.update_layout(
        title=dict(
            text=f"Top {role_name}s | Min {min_minutes} mins" + (f" Under {max_age}" if max_age else ""),
            x=0.5,
            font=dict(size=20, family=FONT_FAMILY, color='#333333'),
            xanchor='center'
        ),
        plot_bgcolor='#f1ffcd',
        paper_bgcolor='#f1ffcd',
        xaxis=dict(
            title='Role Suitability Score (0-100)',
            range=[0, 100],
            showgrid=True,
            gridcolor='#f1ffcd',
            tickfont=dict(family=FONT_FAMILY),
        ),
        yaxis=dict(
            autorange='reversed',
            showgrid=False,
            tickfont=dict(family=FONT_FAMILY, size=12)
        ),
        margin=dict(l=120, r=50, t=80, b=50),
        height=600,
        width=880,
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family=FONT_FAMILY
        )
    )

    stats_output = widgets.Output()
    ipy_display(fig)
    ipy_display(stats_output)

    def on_bar_click(trace, points, selector):
        stats_output.clear_output()
        if not points.point_inds:
            return
        i = points.point_inds[0]
        df_idx = int(trace.customdata[i])
        row = top_players.loc[df_idx]
        lines = [
            f"{row['Player']} (Age: {row.get('Age', '?')}, Club: {row.get('Squad', '?')}, Minutes: {row.get('Mins', '?')})",
            f"Role: {role_name} | Role Score: {row['RoleScore']:.1f}/100"
        ]
        for stat in role_stats:
            val = row.get(stat, np.nan)
            pctl = position_relative_percentile(filtered_df, row, stat)
            if pd.notnull(val):
                lines.append(f"{stat_display_names.get(stat, stat)}: {val:.2f} (Pos Percentile: {pctl:.2f}%)")
            else:
                lines.append(f"{stat_display_names.get(stat, stat)}: N/A")
        lines.append("\nData generated by @nstntly")
        summary_text = "\n".join(lines)
        with stats_output:
            textarea = widgets.Textarea(
                value=summary_text,
                description='',
                layout=widgets.Layout(width='420px', height='320px')
            )
            button = widgets.Button(description="Copy to clipboard", button_style='success')
            js = f"""navigator.clipboard.writeText(`{summary_text}`);"""
            def on_button_click(b):
                try:
                    display(Javascript(js))
                    print("Copied to clipboard!")
                except Exception:
                    print("Copy manually from the box above.")
            button.on_click(on_button_click)
            display(widgets.HBox([textarea, button]))

    fig.data[0].on_click(on_bar_click)

    # Save buttons as before
    output = widgets.Output()
    def save_image(_):
        filename = f"top_{role_name.replace(' ', '_')}_players.png"
        if max_age:
            filename = f"top_{max_age}_under_{role_name.replace(' ', '_')}_players.png"
        try:
            fig.write_image(filename, scale=2)
            with output:
                print(f"Saved image to {filename}")
        except Exception as e:
            with output:
                print(f"Error saving image: {str(e)}")
                print("Please install Kaleido with: pip install kaleido")
    def save_csv(_):
        filename = f"top_{role_name.replace(' ', '_')}_players.csv"
        if max_age:
            filename = f"top_{max_age}_under_{role_name.replace(' ', '_')}_players.csv"
        csv_data = top_players[['Player', 'Age', 'Squad', 'Nation', 'Mins', 'RoleScore'] + role_stats].copy()
        csv_data['Role'] = role_name
        csv_data = csv_data.sort_values('RoleScore', ascending=False)
        csv_data.to_csv(filename, index=False)
        with output:
            print(f"Saved to {filename}")
    image_button = widgets.Button(description="Save as Image", style=dict(button_color='#1A78CF'))
    image_button.on_click(save_image)
    csv_button = widgets.Button(description="Export to CSV", style=dict(button_color='#44C3A1'))
    csv_button.on_click(save_csv)
    ipy_display(widgets.HBox([image_button, csv_button]))
    ipy_display(output)

##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################
##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################
def analyze_player_roles(
    df, archetype_params, stat_display_names, player_row, selected_positions, font_family="Montserrat"
):
    # --- Fix: ensure font_family is always a string, never int or float ---
    if not isinstance(font_family, str) or font_family.isnumeric():
        font_family = "Montserrat"

    name = player_row.get('Player', 'Unknown Player')
    league = player_row.get('Comp', 'Unknown League')
    try:
        age = float(player_row.get('Age', ''))
    except Exception:
        age = ''
    mins = player_row.get("Mins", 0)

    title_text = (
        f"<b style='font-size:1.4em;font-family:{font_family};'>{name}</b> &middot; "
        f"Age: {age if age else '-'} &middot; "
        f"League: {league} &middot; "
        f"Minutes: {mins}"
    )

    # --- Determine this player's relevant position groups (e.g. DM, CB)
    pos_str = player_row.get('Pos', '')
    player_positions = [p.strip() for p in str(pos_str).split(',')] if isinstance(pos_str, str) else []

    def filter_df_by_positions(pos_list, db):
        return db[db['Pos'].apply(
            lambda s: any(pos in str(s).split(',') for pos in pos_list) if isinstance(s, str) else False
        )]

    position_to_roles = {
        "GK": ["GK"],
        "DF": ["CB", "FB", "WB", "LB", "RB", "LWB", "RWB", "SW"],
        "MF": ["DM", "CM", "AM", "LM", "RM", "LW", "RW"],
        "FW": ["ST", "CF", "WF", "Winger"]
    }
    if not selected_positions:
        selected_positions = player_positions

    relevant_roles = []
    for pos_group in selected_positions:
        for k, v in position_to_roles.items():
            if pos_group in v or pos_group == k:
                relevant_roles.extend([role for role in archetype_params if any(kw in role for kw in v)])
    relevant_roles = list(dict.fromkeys(relevant_roles))
    if not relevant_roles:
        print(f"No roles found for position groups: {', '.join(selected_positions)}")
        return

    role_data = []
    full_stats = {}
    pos_filtered_db = filter_df_by_positions(player_positions, df)
    for role in relevant_roles:
        role_stats = archetype_params.get(role, [])
        if role_stats:
            score = calculate_role_score(player_row, role_stats, pos_filtered_db, role)
            stat_details = []
            for stat in role_stats:
                val = player_row.get(stat, np.nan)
                if pd.notnull(val):
                    stat_db = pos_filtered_db[stat].replace([np.inf, -np.inf], np.nan).dropna()
                    if not stat_db.empty:
                        pctl = stats.percentileofscore(stat_db, val, kind='mean')
                        stat_details.append((stat, val, round(pctl, 2)))
            role_data.append({'role': role, 'score': score})
            full_stats[role] = stat_details

    role_data = sorted(role_data, key=lambda x: x['score'], reverse=True)
    if not role_data:
        display(HTML("<b>No role data available for this player.</b>"))
        return

    scores = [d['score'] for d in role_data]
    min_score, max_score = min(scores), max(scores)
    colorscale = 'Blues'
    def get_bar_color(score):
        rel = 0.2 + 0.8 * (score - min_score) / (max_score - min_score + 1e-6)
        return sample_colorscale(colorscale, [rel])[0]
    bar_colors = []
    bar_text_colours = []
    for d in role_data:
        c = get_bar_color(d['score'])
        bar_colors.append(c)
        if c.startswith('rgb('):
            rgb = tuple(map(int, c[4:-1].split(',')))
        elif c.startswith('#'):
            rgb = tuple(int(c[i:i+2], 16) for i in (1, 3, 5))
        else:
            rgb = (0, 0, 0)
        luminance = 0.299*rgb[0] + 0.587*rgb[1] + 0.114*rgb[2]
        bar_text_colours.append("black" if luminance > 170 else "white")

    # --- Make Plotly chart ---
    import plotly.graph_objs as go
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=[d['score'] for d in role_data],
        y=[d['role'] for d in role_data],
        orientation='h',
        marker=dict(color=bar_colors, line=dict(color='rgba(0,0,139,0.2)', width=1)),
        text=[f"{d['score']:.1f}" for d in role_data],
        textposition='inside',
        textfont=dict(size=14, family=font_family, color=bar_text_colours),
        insidetextanchor="middle",
        customdata=[d['role'] for d in role_data],
        hovertemplate="<b>%{y}</b><br>Suitability Score: %{x:.1f}<extra></extra>",
        width=0.58,
    ))

    fig.update_layout(
        title=dict(
            text=f"<b>Role Suitability Analysis</b><br><span style='font-size:1.05em'>{name} | Age: {age if age else '-'} | League: {league}</span>",
            x=0.02, y=0.96, xanchor='left', yanchor='top',
            font=dict(family=font_family, size=18, color="#111")
        ),
        xaxis=dict(
            title="Suitability Score (0-100)",
            range=[0, 100],
            showgrid=False,
            title_font=dict(size=18, family=font_family, color="#444"),
            tickfont=dict(size=16, family=font_family, color="#222")
        ),
        yaxis=dict(
            autorange='reversed',
            showgrid=False,
            title=None,
            tickfont=dict(size=17, family=font_family, color="#222")
        ),
        height=60 + 46 * len(role_data),
        width=600,
        margin=dict(l=130, r=30, t=100, b=50),
        plot_bgcolor='#f1ffcd',
        paper_bgcolor='#f1ffcd',
        showlegend=False,
        hoverlabel=dict(bgcolor='white', font_size=13, font_family=font_family),
    )

    # --- Output area and click logic unchanged (small font tweaks inside) ---
    stats_output = widgets.Output(layout={'border': '1px solid #dde', 'margin': '12px 0'})
    copy_output = widgets.Output(layout={'border': '1px solid #dde', 'margin': '6px 0'})
    last_clicked = {'role': None}

    def on_click(trace, points, selector):
        stats_output.clear_output()
        copy_output.clear_output()
        if not points.point_inds:
            return
        ind = points.point_inds[0]
        role = role_data[ind]['role']
        stat_tuples = full_stats.get(role, [])
        with stats_output:
            display(HTML(
                f"<h4 style='margin-bottom:4px;color:#1A78CF;font-family:{font_family};font-size:1.15em'>{role}</h4>"
                f"<div style='margin-bottom:7px;font-size:0.98em'>Overall Score: <b>{role_data[ind]['score']:.1f}/100</b></div>"
            ))
            if not stat_tuples:
                display(HTML("<i>No stat details for this role.</i>"))
            else:
                grid_html = "<table style='border-collapse:collapse;font-size:12.5px'>"
                grid_html += "<tr><th style='text-align:left;padding-right:8px'>Stat</th><th style='text-align:right'>Value</th><th style='text-align:right'>Percentile</th></tr>"
                for stat, val, pctl in stat_tuples:
                    stat_name = stat_display_names.get(stat, stat)
                    grid_html += f"<tr><td style='padding-right:8px'>{stat_name}</td><td style='text-align:right'>{val:.2f}</td><td style='text-align:right'>{pctl:.1f}%</td></tr>"
                grid_html += "</table>"
                display(HTML(grid_html))
        with copy_output:
            meta_str = f"{name} (Age: {age if age else '-'}, League: {league}, Minutes: {mins})"
            stat_lines = "\n".join([
                f"{stat_display_names.get(s[0], s[0])}: {s[1]:.2f} ({s[2]:.1f}%)"
                for s in stat_tuples
            ])
            txt = f"{meta_str}\nRole: {role}\nScore: {role_data[ind]['score']:.1f}/100\n\n{stat_lines}"
            textarea = widgets.Textarea(value=txt, layout={'width': '98%', 'height': '125px'})
            button = widgets.Button(description="Copy", button_style='success')
            js = f"""navigator.clipboard.writeText(`{txt}`);"""
            def on_button_click(b):
                try:
                    display(Javascript(js))
                    print("Copied to clipboard!")
                except:
                    print("Copy manually from the text box.")
            button.on_click(on_button_click)
            display(widgets.VBox([textarea, button]))
        last_clicked['role'] = role

    try:
        fig.data[0].on_click(on_click)
    except Exception:
        pass

    download_btn = widgets.Button(
        description="Download PNG", 
        button_style='info',
        layout={'width': '110px', 'margin': '8px 0'}
    )
    download_status = widgets.Output()

    def on_download_click(b):
        with download_status:
            try:
                filename = f"{name.replace(' ', '_')}_role_analysis.png"
                filename = re.sub(r'[^\w\-_.]', '', filename)
                buf = BytesIO()
                fig.write_image(buf, format='png', width=800, height=600, scale=4)
                b64 = base64.b64encode(buf.getvalue()).decode()
                html = f'<a href="data:image/png;base64,{b64}" download="{filename}">Click to download</a>'
                display(HTML(html))
                print(f"Saved as {filename}")
            except Exception as e:
                print(f"Error saving image: {str(e)}")
                print("Please install kaleido with: pip install kaleido")

    download_btn.on_click(on_download_click)

    display(HTML(title_text))
    display(fig)
    display(widgets.HBox([download_btn]))
    display(download_status)
    display(HTML(
        "<div style='margin:6px 0;color:#666;font-size:13px'>"
        "<b>Click any bar</b> for stat details by role (percentiles vs same-position players only).</div>"
    ))
    display(stats_output)
    display(copy_output)

    # Auto-select first role
    if role_data:
        fake_points = type("Points", (), {"point_inds": [0]})()
        on_click(fig.data[0], fake_points, None)



#this is probs broken, don't worry, i'll fix at some point

def show_radar_chart(player_row, stat_cols, stat_labels, df):
    # Calculate percentiles for each stat (position-relative)
    percentiles = [
        position_relative_percentile(df, player_row, stat) if np.isfinite(player_row.get(stat, np.nan)) else 0
        for stat in stat_cols
    ]
    chosen_role = archetype_keys[arch_idx]
    # Radar plot logic (leave all formatting as is)
    values = percentiles + percentiles[:1]
    N = len(stat_cols)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    angles += angles[:1]

    fig, ax = plt.subplots(figsize=(9, 11), subplot_kw=dict(polar=True), facecolor="#212435")
    fig.patch.set_facecolor('#222222')
    ax.set_facecolor('#222222')
    fig.subplots_adjust(top=0.78, bottom=0.08)

    ax.plot(angles, values, color="#4da6ff", linewidth=3, zorder=2)
    ax.fill(angles, values, color="#4da6ff", alpha=0.19, zorder=2)
    ax.scatter(angles, values, color="#fff", s=90, zorder=3, edgecolor="#1A78CF", linewidth=2)

    label_radius = 138
    for angle, label in zip(angles[:-1], stat_labels):
        ax.text(
            angle, label_radius, label,
            size=18, ha='center', va='center',
            color="#e1e9fd", fontproperties=font_normal, rotation=0,
            clip_on=False, zorder=10
        )

    ax.set_ylim(0, 100)
    ax.set_yticks([20, 40, 60, 80, 100])
    ax.set_yticklabels([str(x) for x in [20, 40, 60, 80, 100]], color="#8ca6d6", fontsize=13, fontproperties=font_normal)
    ax.yaxis.grid(True, color="#405278", linewidth=1.3, alpha=0.4)
    ax.xaxis.grid(False)
    ax.spines["polar"].set_color("#405278")
    ax.spines["polar"].set_linewidth(1.4)
    ax.set_xticks([])

    player_name = player_row.get('Player', 'Unknown Player')
    fig.text(0.5, 0.97, player_name, ha='center', va='top',
             fontsize=36, color="#e1e9fd", fontproperties=font_normal)
    fig.text(0.5, 0.930, chosen_role, ha='center', va='top',
             fontsize=22, color="#e1e9fd", fontproperties=font_normal, alpha=0.93)
    fig.text(0.5, 0.900, "in percentiles", ha='center', va='top', fontsize=16, color='#e1e9fd', fontproperties=font_normal, alpha=0.82)

    clean_role = chosen_role
    save_name = f"{player_name.replace(' ', '_')}_{clean_role}_radar.png"
    plt.savefig(save_name, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.show()
    print(f"Radar plot saved as {save_name}")


##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################
##############################################################################################################################
############################################################################################################################
##############################################################################################################################
################################################################################################################################



def pizza_no_toppings(player_row, stat_cols, stat_labels, df):
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    raw_stat_values = [player_row.get(s, float('nan')) for s in stat_cols]
    chosen_role = archetype_keys[arch_idx]
    # Calculate role score using consistent method
    role_score = calculate_role_score(player_row, role_stats, df, role_name)
    
    # Calculate percentiles for each stat (position-relative)
    percentiles = [
        position_relative_percentile(df, player_row, stat) if np.isfinite(player_row.get(stat, np.nan)) else 0
        for stat in stat_cols
    ]
    def get_contrast_text_color(hex_color):
        r, g, b = mcolors.hex2color(hex_color)
        brightness = (r*299 + g*587 + b*114) * 255 / 1000
        return "#000000" if brightness > 140 else "#F2F2F2"

    slice_colors = [category_by_param.get(s, "#2E4374") for s in stat_cols]
    text_colors = [get_contrast_text_color(c) for c in slice_colors] 
    
    
    def break_label(label, max_len=15):
        # Handle labels with /90 at the end
        if label.endswith("/90"):
            before = label[:-3].rstrip()  # Remove '/90'
            if len(label) <= max_len:
                return label  # No need to break
            words = before.split(" ")
            lines = []
            current_line = ""
            for word in words:
                # Check if adding word would exceed max_len (accounting for /90 at end)
                check_len = len(current_line + (" " if current_line else "") + word + "/90")
                if len(lines) == 0 and check_len > max_len:
                    # If the first line would exceed max_len, break here
                    if current_line:
                        lines.append(current_line)
                    current_line = word
                elif len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            # Add last line + /90
            lines.append(current_line + "/90")
            return "\n".join(lines)
        else:
            # For labels without /90, break as usual if needed
            if len(label) <= max_len:
                return label
            words = label.split(" ")
            lines = []
            current_line = ""
            for word in words:
                if len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            if current_line:
                lines.append(current_line)
            return "\n".join(lines)


    display_params = [break_label(stat_display_names.get(p, p), max_len=15) for p in stat_cols]

    
    

    figsize = (10, 11)
    baker = PyPizza(
        params=display_params,
        background_color="#222222",
        straight_line_color="#000000",
        straight_line_lw=.3,
        last_circle_color="#000000",
        last_circle_lw=1,
        other_circle_lw=0,
        inner_circle_size=0.3
    )
    fig, ax = baker.make_pizza(
        percentiles,
        alt_text_values=raw_stat_values,
        figsize=figsize,
        color_blank_space="same",
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(
            edgecolor="#000000", zorder=2, linewidth=1
        ),
        kwargs_params=dict(
            color="#fffff0", fontsize=18,
            fontproperties=font_normal, va="center"
        ),
        kwargs_values=dict(
            color="#fffff0", fontsize=15,
            fontproperties=font_normal, zorder=3,
            bbox=dict(
                edgecolor="#000000", facecolor="cornflowerblue",
                boxstyle="round,pad=0.2", lw=1
            )
        )
    )
    save_name = f"{player_name.replace(' ', '_')}_{chosen_role}_pizza_no_toppings.png"
    plt.savefig(save_name, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.show()
    print("Saved!")

def pizza_no_toppings_lightmode(player_row, stat_cols, stat_labels, df, min_minutes):
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    raw_stat_values = [player_row.get(s, float('nan')) for s in stat_cols]
    chosen_role = archetype_keys[arch_idx]
    # Calculate role score using consistent method
    role_score = calculate_role_score(player_row, role_stats, df, role_name)
    
    # Calculate percentiles for each stat (position-relative)
    percentiles = [
        position_relative_percentile(df, player_row, stat) if np.isfinite(player_row.get(stat, np.nan)) else 0
        for stat in stat_cols
    ]
    def get_contrast_text_color(hex_color):
        r, g, b = mcolors.hex2color(hex_color)
        brightness = (r*299 + g*587 + b*114) * 255 / 1000
        return "#000000" if brightness > 140 else "#F2F2F2"

    slice_colors = [category_by_param.get(s, "#2E4374") for s in stat_cols]
    text_colors = [get_contrast_text_color(c) for c in slice_colors] 
    
    
    def break_label(label, max_len=15):
        # Handle labels with /90 at the end
        if label.endswith("/90"):
            before = label[:-3].rstrip()  # Remove '/90'
            if len(label) <= max_len:
                return label  # No need to break
            words = before.split(" ")
            lines = []
            current_line = ""
            for word in words:
                # Check if adding word would exceed max_len (accounting for /90 at end)
                check_len = len(current_line + (" " if current_line else "") + word + "/90")
                if len(lines) == 0 and check_len > max_len:
                    # If the first line would exceed max_len, break here
                    if current_line:
                        lines.append(current_line)
                    current_line = word
                elif len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            # Add last line + /90
            lines.append(current_line + "/90")
            return "\n".join(lines)
        else:
            # For labels without /90, break as usual if needed
            if len(label) <= max_len:
                return label
            words = label.split(" ")
            lines = []
            current_line = ""
            for word in words:
                if len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            if current_line:
                lines.append(current_line)
            return "\n".join(lines)


    display_params = [break_label(stat_display_names.get(p, p), max_len=15) for p in stat_cols]

    
    

    figsize = (10, 11)
    baker = PyPizza(
        params=display_params,
        background_color="#f1ffcd",
        straight_line_color="#000000",
        straight_line_lw=.3,
        last_circle_color="#000000",
        last_circle_lw=1,
        other_circle_lw=0,
        inner_circle_size=0.3
    )
    fig, ax = baker.make_pizza(
        percentiles,
        alt_text_values=raw_stat_values,
        figsize=figsize,
        color_blank_space="same",
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(
            edgecolor="#000000", zorder=2, linewidth=1
        ),
        kwargs_params=dict(
            color="#222222", fontsize=18,
            fontproperties=font_normal, va="center"
        ),
        kwargs_values=dict(
            color="#222222", fontsize=15,
            fontproperties=font_normal, zorder=3,
            bbox=dict(
                edgecolor="#000000", facecolor="cornflowerblue",
                boxstyle="round,pad=0.2", lw=1
            )
        )
    )
    save_name = f"{player_name.replace(' ', '_')}_{chosen_role}_pizza_no_toppings_lightmode.png"
    plt.savefig(save_name, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.show()
    print("Saved!")

def show_pizza_chart_lightmode(player_row, stat_cols, stat_labels, df, min_minutes):
    role_name = archetype_keys[arch_idx]
    role_stats = archetype_params.get(role_name, stat_cols)
    raw_stat_values = [player_row.get(s, float('nan')) for s in stat_cols]
    
    # Calculate role score using consistent method
    role_score = calculate_role_score(player_row, role_stats, df, role_name)
    
    # Calculate percentiles for each stat (position-relative)
    percentiles = [
        position_relative_percentile(df, player_row, stat) if np.isfinite(player_row.get(stat, np.nan)) else 0
        for stat in stat_cols
    ]
    
        # --- DYNAMIC CONTRAST COLOUR LOGIC ---
    def get_contrast_text_color(hex_color):
        r, g, b = mcolors.hex2color(hex_color)
        brightness = (r*299 + g*587 + b*114) * 255 / 1000
        return "#000000" if brightness > 140 else "#F2F2F2"

    slice_colors = [category_by_param.get(s, "#2E4374") for s in stat_cols]
    text_colors = [get_contrast_text_color(c) for c in slice_colors]

    def break_label(label, max_len=15):
        # Handle labels with /90 at the end
        if label.endswith("/90"):
            before = label[:-3].rstrip()  # Remove '/90'
            if len(label) <= max_len:
                return label  # No need to break
            words = before.split(" ")
            lines = []
            current_line = ""
            for word in words:
                # Check if adding word would exceed max_len (accounting for /90 at end)
                check_len = len(current_line + (" " if current_line else "") + word + "/90")
                if len(lines) == 0 and check_len > max_len:
                    # If the first line would exceed max_len, break here
                    if current_line:
                        lines.append(current_line)
                    current_line = word
                elif len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            # Add last line + /90
            lines.append(current_line + "/90")
            return "\n".join(lines)
        else:
            # For labels without /90, break as usual if needed
            if len(label) <= max_len:
                return label
            words = label.split(" ")
            lines = []
            current_line = ""
            for word in words:
                if len(current_line + (" " if current_line else "") + word) > max_len:
                    lines.append(current_line)
                    current_line = word
                else:
                    current_line += (" " if current_line else "") + word
            if current_line:
                lines.append(current_line)
            return "\n".join(lines)


    display_params = [break_label(stat_display_names.get(p, p), max_len=15) for p in stat_cols]

    figsize = (10, 11)
    baker = PyPizza(
        params=display_params,
        background_color="#f1ffcd",
        straight_line_color="#000000",
        straight_line_lw=.3,
        last_circle_color="#000000",
        last_circle_lw=1,
        other_circle_lw=0,
        inner_circle_size=0.3
    )
    fig, ax = baker.make_pizza(
        percentiles,
        alt_text_values=raw_stat_values,
        figsize=figsize,
        color_blank_space="same",
        slice_colors=slice_colors,
        value_colors=text_colors,
        value_bck_colors=slice_colors,
        blank_alpha=0.4,
        kwargs_slices=dict(
            edgecolor="#000000", zorder=2, linewidth=1
        ),
        kwargs_params=dict(
            color="#222222", fontsize=18,
            fontproperties=font_normal, va="center"
        ),
        kwargs_values=dict(
            color="#222222", fontsize=15,
            fontproperties=font_normal, zorder=3,
            bbox=dict(
                edgecolor="#000000", facecolor="cornflowerblue",
                boxstyle="round,pad=0.2", lw=1
            )
        )
    )
    

    
    club = player_row.get("Squad", "?")
    nationality = player_row.get("Nation", "?")
    age = player_row.get("Age", "?")
    mins_played = player_row.get("Mins", 0)
    season = "2024/25"
    chosen_role = archetype_keys[arch_idx]
    if isinstance(selected_positions, list):
        positions_label = ", ".join(str(pos) for pos in selected_positions)
    else:
        positions_label = str(selected_positions)
    
    #player name and heading
    
    fig.text(
        0.5, 0.985, player_row['Player'], ha='center', va='top', fontsize=22,
        fontweight='bold', color='#222222', fontproperties=font_normal
    )
    fig.text(
        0.5, 0.952, f"{club} | {nationality} | {age} | {season}", ha='center', va='top', fontsize=15,
        color='#222222', fontproperties=font_normal
    )
    fig.text(
        0.5, 0.928, f"Compared vs: {positions_label} | Role: {chosen_role} | Minutes played: {mins_played}",
        ha='center', va='top', fontsize=12, color='#222222', fontproperties=font_normal
    )
    # this text is the credit text at the bottom
    fig.text(
        0.5, 0.01, "Created by @nstntly | Inspired by @booteful_game",
        ha='center', va='bottom', fontsize=9, color='#CCCCCC', fontproperties=font_normal, alpha=0.8
    )

    
    # role score attachment

    fig.text(
        0.845, 0.9, f"Role Score: {role_score:.1f}", 
        ha="center", va="top",
        fontsize=14,
        fontproperties=font_normal,
        fontweight="bold",
        color="#222222",
        bbox=dict(
            boxstyle="round,pad=0.1",
            facecolor="#1A78CF" if role_score >= 70 else "#D70232" if role_score < 40 else "#FF9300",
            edgecolor="#000000",
            linewidth=1
        ),
        zorder=5
    )
    # save logic, saves to whatever folder the program is in!
    save_name = f"pizza_{player_row['Player'].replace(' ', '_')}_{season.replace('/', '-')}_lightmode.png"
    plt.savefig(save_name, dpi=300, bbox_inches='tight', facecolor=fig.get_facecolor())
    plt.show()
    print(f"Pizza plot saved as {save_name}")


def calculate_player_synergy(player_row, df, archetype_params, stat_display_names, font_family="Montserrat", top_n=10, score_band=7):
    name = player_row.get('Player', 'Unknown Player')
    league = player_row.get('Comp', 'Unknown League')
    try:
        age = float(player_row.get('Age', ''))
    except Exception:
        age = ''
    mins = player_row.get("Mins", 0)


    # --- Determine this player's relevant position groups (e.g. DM, CB)
    pos_str = player_row.get('Pos', '')
    player_positions = [p.strip() for p in str(pos_str).split(',')] if isinstance(pos_str, str) else []

    # --- Filter DB for players in any of the same positions as the player
    def filter_df_by_positions(pos_list, db):
        # Returns dataframe with at least one of pos_list in its Pos field (comma-separated)
        return db[db['Pos'].apply(
            lambda s: any(pos in str(s).split(',') for pos in pos_list) if isinstance(s, str) else False
        )]

    position_to_roles = {
        "GK": ["GK"],
        "DF": ["CB", "FB", "WB", "LB", "RB", "LWB", "RWB", "SW"],
        "MF": ["DM", "CM", "AM", "LM", "RM", "LW", "RW"],
        "FW": ["ST", "CF", "WF", "Winger"]
    }
    # Allow all roles if no selected_positions provided (rare)
    if not selected_positions:
        selected_positions = player_positions

    relevant_roles = []
    for pos_group in selected_positions:
        for k, v in position_to_roles.items():
            if pos_group in v or pos_group == k:
                relevant_roles.extend([role for role in archetype_params if any(kw in role for kw in v)])
    # Remove duplicates
    relevant_roles = list(dict.fromkeys(relevant_roles))
    if not relevant_roles:
        print(f"No roles found for position groups: {', '.join(selected_positions)}")
        return

    # --- For each role, calculate score and stat percentiles using only matching-position DB
    role_data = []
    full_stats = {}
    pos_filtered_db = filter_df_by_positions(player_positions, df)
    for role in relevant_roles:
        role_stats = archetype_params.get(role, [])
        if role_stats:
            # Calculate role score using filtered DB (so percentileofscore is fair)
            score = calculate_role_score(player_row, role_stats, pos_filtered_db, role)
            stat_details = []
            for stat in role_stats:
                val = player_row.get(stat, np.nan)
                if pd.notnull(val):
                    # Percentile among matching positions
                    stat_db = pos_filtered_db[stat].replace([np.inf, -np.inf], np.nan).dropna()
                    if not stat_db.empty:
                        pctl = stats.percentileofscore(stat_db, val, kind='mean')
                        stat_details.append((stat, val, round(pctl, 2)))
            role_data.append({'role': role, 'score': score})
            full_stats[role] = stat_details

    role_data = sorted(role_data, key=lambda x: x['score'], reverse=True)
    if not role_data:
        display(HTML("<b>No role data available for this player.</b>"))
        return

    # 3. Find all other players in this group
    group_players = df[df['Pos'].str.contains(broad_pos, na=False)]
    print(f"Num players in group: {len(group_players)}")
    
    # 4. For each, calculate score in your weakest role
    partners = []
    for _, row in group_players.iterrows():
        if row['Player'] == player_row['Player']:
            continue
        score = calculate_role_score(row, archetype_params[worst_role], df, worst_role)
        if abs(score - best_score) <= score_band:
            partners.append({
                'Player': row['Player'],
                'Team': row.get('Squad', ''),
                'Age': row.get('Age', ''),
                'Role': worst_role,
                'Score': score,
                'Distance': abs(score - best_score)
            })

    if not partners:
        print("No synergy partners found")
        return

    # 5. Take top_n by closest score (smallest distance, then highest score)
    partners = sorted(partners, key=lambda x: (x['Distance'], -x['Score']))[:top_n]
    
    # 6. Visualise
    labels = [f"{p['Player']} ({p['Team']}, {p['Age']})\n{p['Role']} ({p['Score']:.1f})" for p in partners]
    scores = [p['Score'] for p in partners]
    fig, ax = plt.subplots(figsize=(12, 8))
    bars = ax.barh(labels, scores, color='#1fa44b')
    ax.set_xlabel("Role Score", fontsize=13, fontname=font_family)
    ax.set_title(
        f"Top Synergy Partners for {player_row['Player']}\n"
        f"Best Role: {best_role} ({best_score:.1f}) | Synergy Role: {worst_role}",
        fontsize=15, fontname=font_family)
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.7, bar.get_y() + bar.get_height()/2,
                f"{width:.1f}", ha='left', va='center', fontsize=11, fontname=font_family)
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

    # Also print details
    print("\nTop Synergy Partners:")
    for i, p in enumerate(partners, 1):
        print(f"{i}. {p['Player']} ({p['Team']}, Age {p['Age']}) - {p['Role']}: {p['Score']:.1f}")

def show_top_players_by_stat(df, stat_display_names, min_minutes=None, selected_positions=None):
    from IPython.display import display as ipy_display
    import ipywidgets as widgets
    import matplotlib.colors as mcolors
    import plotly.graph_objs as go

    FONT_FAMILY = "Gabarito, Montserrat, Arial, sans-serif"

    # ---- Get possible stat columns (numeric) ----
    possible_stats = [
        c for c in df.columns
        if pd.api.types.is_numeric_dtype(df[c]) and c not in ['Age', 'Mins', 'Squad', 'Player', 'Comp', 'Nation', 'Pos']
    ]
    display_to_raw = {stat_display_names.get(col, col).lower(): col for col in possible_stats}
    raw_to_display = {col: stat_display_names.get(col, col) for col in possible_stats}
    display_names_list = [stat_display_names.get(c, c) for c in possible_stats]

    print("\nAvailable stats to rank by:")
    for i, display_name in enumerate(display_names_list, 1):
        print(f"{i:2d}. {display_name}")

    # --- Prompt for stat by display name ---
    while True:
        stat_disp_input = input("\nEnter the stat name to rank by (case-insensitive, display name): ").strip().lower()
        matches = [d for d in display_names_list if stat_disp_input in d.lower()]
        if len(matches) == 1:
            stat_col = display_to_raw[matches[0].lower()]
            tidy_label = matches[0]
            break
        elif len(matches) > 1:
            print("Multiple matches found:")
            for m in matches:
                print(f"- {m}")
            continue
        else:
            print("No matches found, please try again.")

    # --- Min minutes input ---
    if min_minutes is None:
        min_minutes = input("Enter minimum minutes played (default 900): ").strip()
        min_minutes = int(min_minutes) if min_minutes else 900
    df = df.copy()
    df['Mins'] = pd.to_numeric(df['Mins'], errors='coerce')
    filtered_df = df[df['Mins'] >= min_minutes]

    # --- League filter ---
    def strip_main_prefix(comp):
        prefixes = ('eng ', 'it ', 'es ', 'de ', 'fr ')
        for prefix in prefixes:
            if comp.lower().startswith(prefix):
                return comp[len(prefix):]
        return comp

    filtered_df['LeagueName'] = filtered_df['Comp'].astype(str).apply(strip_main_prefix)
    leagues = filtered_df['LeagueName'].unique()
    print("\nAvailable leagues loaded:")
    print(", ".join(leagues))
    league_input = input("Filter by league(s)? (comma sep, blank for all): ").strip()
    if league_input:
        chosen_leagues = [l.strip().lower() for l in league_input.split(",") if l.strip()]
        mask = filtered_df['LeagueName'].str.lower().isin(chosen_leagues)
        if not mask.any():
            print("No matches found for input! Please check league names as shown above.")
            return
        filtered_df = filtered_df[mask]
        print(f"Players after league filter: {len(filtered_df)}")
    else:
        print("No league filter applied.")
    filtered_df = filtered_df.drop(columns=["LeagueName"])

    # --- Top 10 by stat_col ---
    top_players = filtered_df.nlargest(10, stat_col).reset_index(drop=True)

    # --- Blue gradient as in leaderboard ---
    blue_gradient = [
        "#B2DFFC", "#8EC3E0", "#6FA7D6", "#578BC8", "#4370BA",
        "#3457A4", "#274097", "#1F2C7B", "#1A3474", "#112052"
    ][::-1]
    bar_colors = blue_gradient[-len(top_players):][::-1] if len(top_players) <= 10 else \
        [blue_gradient[0] for _ in range(len(top_players))]
    def luminance(hexcolor):
        rgb = mcolors.hex2color(hexcolor)
        r, g, b = [x * 255 for x in rgb]
        return 0.299*r + 0.587*g + 0.114*b
    bar_text_colors = ['white' if luminance(c) < 150 else '#222' for c in bar_colors]

    # --- Bar labels ---
    bar_labels = []
    for _, row in top_players.iterrows():
        bar_labels.append(
            f"{row['Player']} • {row.get(stat_col, 0):.2f} • {row.get('Age', '?')} • {row.get('Squad', '?')} • {int(row.get('Mins',0)):,} mins"
        )

    # --- Position label for title ---
    if selected_positions:
        pos_label = ", ".join(selected_positions)
    else:
        pos_label = "All positions"

    # --- Bar chart setup
    fig = go.FigureWidget([
        go.Bar(
            x=top_players[stat_col],
            y=[f"#{i+1}" for i in range(len(top_players))],
            orientation='h',
            text=bar_labels,
            textposition='inside',
            insidetextanchor='middle',
            marker=dict(
                color=bar_colors,
                line=dict(color='#333333', width=1)
            ),
            textfont=dict(
                color=bar_text_colors,
                size=13,
                family=FONT_FAMILY
            ),
            customdata=top_players.index.values,
            hovertext=[f"{row['Player']} ({row['Squad']})" for _, row in top_players.iterrows()],
            hoverinfo="text"
        )
    ])

    fig.update_layout(
        title=dict(
            text=f"Top 10: {tidy_label} | Min {min_minutes} mins | {pos_label}",
            x=0.5,
            font=dict(size=20, family=FONT_FAMILY, color='#333333'),
            xanchor='center'
        ),
        plot_bgcolor='#f1ffcd',
        paper_bgcolor='#f1ffcd',
        xaxis=dict(
            title=f"{tidy_label}",
            showgrid=True,
            gridcolor='#f1ffcd',
            tickfont=dict(family=FONT_FAMILY),
        ),
        yaxis=dict(
            autorange='reversed',
            showgrid=False,
            tickfont=dict(family=FONT_FAMILY, size=12)
        ),
        margin=dict(l=120, r=50, t=80, b=50),
        height=600,
        width=880,
        hoverlabel=dict(
            bgcolor='white',
            font_size=12,
            font_family=FONT_FAMILY
        )
    )
    ipy_display(fig)

def show_stat_scatter(df, stat_display_names, selected_positions=None, outlier_pct=1.0):
    import plotly.graph_objs as go
    import numpy as np
    from IPython.display import display as ipy_display
    import ipywidgets as widgets

    FONT_FAMILY = "Gabarito, Montserrat, Arial, sans-serif"

    # --- Stat selection logic (as before) ---
    possible_stats = [
        c for c in df.columns
        if pd.api.types.is_numeric_dtype(df[c]) and c not in ['Age', 'Mins', 'Squad', 'Player', 'Comp', 'Nation', 'Pos']
    ]
    display_to_raw = {stat_display_names.get(col, col).lower(): col for col in possible_stats}
    display_names_list = [stat_display_names.get(c, c) for c in possible_stats]

    print("\nAvailable stats:")
    for i, display_name in enumerate(display_names_list, 1):
        print(f"{i:2d}. {display_name}")

    def prompt_stat(display_names_list, prompt_str):
        while True:
            stat_disp_input = input(prompt_str).strip()
            if stat_disp_input.isdigit():
                idx = int(stat_disp_input) - 1
                if 0 <= idx < len(display_names_list):
                    return display_names_list[idx]
                else:
                    print("Number out of range.")
            else:
                matches = [d for d in display_names_list if stat_disp_input.lower() == d.lower()]
                if len(matches) == 1:
                    return matches[0]
                elif len(matches) > 1:
                    print("Multiple exact matches found (shouldn't happen):", matches)
                else:
                    print("No exact match found, please enter number or exact stat name.")

    tidy_x = prompt_stat(display_names_list, "\nEnter the stat for X-axis (number or exact name): ")
    tidy_y = prompt_stat(display_names_list, "Enter the stat for Y-axis (number or exact name): ")
    stat_x = display_to_raw[tidy_x.lower()]
    stat_y = display_to_raw[tidy_y.lower()]

    if selected_positions:
        pos_label = ", ".join(selected_positions)
    else:
        pos_label = "All positions"

    df = df.copy()
    df = df.dropna(subset=[stat_x, stat_y])
    df['Age'] = df['Age'].fillna('?')

    x_vals = df[stat_x]
    y_vals = df[stat_y]

    # Outlier labels: Top 5 and bottom 3 for each stat (unique)
    x_top5 = x_vals.nlargest(5).index
    x_bot3 = x_vals.nsmallest(3).index
    y_top5 = y_vals.nlargest(5).index
    y_bot3 = y_vals.nsmallest(3).index

    outlier_indices = set(x_top5) | set(x_bot3) | set(y_top5) | set(y_bot3)

    # --- Percentiles for hover text ---
    def stat_percentile(col, v):
        vals = df[col].dropna()
        return round((vals < v).mean() * 100, 1)

    hover_texts = [
        f"<b>{row['Player']}</b><br>"
        f"Team: {row.get('Squad', '?')}<br>"
        f"Age: {row.get('Age', '?')}<br>"
        f"{tidy_x}: {row[stat_x]:.2f} (Percentile: {stat_percentile(stat_x, row[stat_x]):.1f}%)<br>"
        f"{tidy_y}: {row[stat_y]:.2f} (Percentile: {stat_percentile(stat_y, row[stat_y]):.1f}%)<br>"
        f"Minutes: {int(row.get('Mins',0)):,}"
        for _, row in df.iterrows()
    ]

    # Name labels: outliers only at first
    name_labels = [
        row['Player'] if idx in outlier_indices else ""
        for idx, row in df.iterrows()
    ]

    # Default marker appearance (normal)
    base_marker = dict(
        size=9,
        color=df['Age'],
        colorscale="Blues",
        opacity=1,
        line=dict(width=1, color="#333")
    )

    # Scatter plot as FigureWidget
    fig = go.FigureWidget(
        data=[
            go.Scatter(
                x=df[stat_x],
                y=df[stat_y],
                mode='markers+text',
                textposition='top center',
                textfont=dict(color="#222222", size=11, family=FONT_FAMILY),
                text=name_labels,
                marker=base_marker,
                hovertext=hover_texts,
                hoverinfo='text'
            )
        ]
    )

    fig.update_layout(
        title=dict(
            text=f"{tidy_x} vs {tidy_y} | {pos_label}",
            x=0.5,
            font=dict(size=20, family=FONT_FAMILY, color='#333333')
        )),
    fig.update_layout(
        xaxis=dict(
            title=dict(
                text=tidy_x,
                font=dict(family=FONT_FAMILY, size=20, color='#333')
            ),
            showgrid=True,
            gridcolor='#f1ffcd',
            tickfont=dict(family=FONT_FAMILY, size=16, color="#222"),
            zeroline=False,
            zerolinewidth=2,
            zerolinecolor='#000',
            linecolor='#000',
            linewidth=2,
            mirror=True
        ),
        yaxis=dict(
            title=dict(
                text=tidy_y,
                font=dict(family=FONT_FAMILY, size=20, color='#333')
            ),
            showgrid=True,
            gridcolor='#f1ffcd',
            tickfont=dict(family=FONT_FAMILY, size=16, color="#222"),
            zeroline=False,
            zerolinewidth=2,
            zerolinecolor='#000',
            linecolor='#000',
            linewidth=2,
            mirror=True
        ),
            plot_bgcolor='#f1ffcd',
            paper_bgcolor='#f1ffcd',
            height=1400,
            width=1960,
            hoverlabel=dict(bgcolor='white', font_size=13, font_family=FONT_FAMILY)
        )

    # ------------- Name search widget ---------------
    name_search = widgets.Text(
        value='', placeholder='Type player name…', description='Highlight:', layout=widgets.Layout(width='350px')
    )

    def update_highlight(change):
        search_val = name_search.value.strip().lower()
        if not search_val or search_val == "reset":
            # Reset to original: all points normal opacity/size/color, outliers have names
            opacities = np.ones(len(df))
            sizes = np.full(len(df), 9)
            border_colors = np.full(len(df), "#333")
            updated_names = [
                row['Player'] if idx in outlier_indices else ""
                for idx, row in df.iterrows()
            ]
        else:
            highlight_mask = df['Player'].str.lower().str.contains(search_val)
            opacities = np.where(highlight_mask, 1, 0.23)
            sizes = np.where(highlight_mask, 22, 9)
            border_colors = np.where(highlight_mask, "#B80019", "#A0A0A0")
            updated_names = [
                row['Player'] if highlight_mask[idx] or idx in outlier_indices else ""
                for idx, row in df.iterrows()
            ]
        with fig.batch_update():
            fig.data[0].marker.opacity = opacities
            fig.data[0].marker.size = sizes
            fig.data[0].marker.line.color = border_colors
            fig.data[0].text = updated_names

    name_search.observe(update_highlight, names='value')

    display(widgets.HBox([name_search]))
    ipy_display(fig)
    update_highlight(None)

def show_role_suitability_scatter(
    df, archetype_params_full, stat_display_names, selected_positions=None
):
    import plotly.graph_objs as go
    import numpy as np
    import ipywidgets as widgets
    from IPython.display import display as ipy_display

    FONT_FAMILY = "Gabarito, Montserrat, Arial, sans-serif"

    # 1. Select roles for X and Y axes
    archetype_keys = list(archetype_params_full.keys())
    print("\nArchetypes:")
    for i, k in enumerate(archetype_keys, 1):
        print(f"{i:2d}. {k}")

    def select_archetype(prompt):
        while True:
            try:
                arch_idx = int(input(prompt)) - 1
                if 0 <= arch_idx < len(archetype_keys):
                    return archetype_keys[arch_idx]
                print(f"Please enter a number between 1 and {len(archetype_keys)}")
            except ValueError:
                print("Please enter a valid number")

    role_x = select_archetype("\nChoose archetype for X-axis by number: ")
    role_y = select_archetype("Choose archetype for Y-axis by number: ")

    print(f"Selected roles: X-axis = {role_x}, Y-axis = {role_y}")

    # 2. Calculate scores for each player
    df = df.copy()
    df['RoleScore_X'] = [
        calculate_role_score(row, archetype_params_full[role_x], df, role_x)
        for _, row in df.iterrows()
    ]
    df['RoleScore_Y'] = [
        calculate_role_score(row, archetype_params_full[role_y], df, role_y)
        for _, row in df.iterrows()
    ]

    # 3. Outlier detection: top 5 and bottom 3 for each axis
    x_vals = df['RoleScore_X']
    y_vals = df['RoleScore_Y']
    x_top5 = x_vals.nlargest(10).index
    x_bot3 = x_vals.nsmallest(10).index
    y_top5 = y_vals.nlargest(10).index
    y_bot3 = y_vals.nsmallest(10).index
    outlier_indices = set(x_top5) | set(x_bot3) | set(y_top5) | set(y_bot3)

    # 4. Hover texts & labels
    hover_texts = [
        f"<b>{row['Player']}</b><br>"
        f"Team: {row.get('Squad', '?')}<br>"
        f"Age: {row.get('Age', '?')}<br>"
        f"{role_x}: {row['RoleScore_X']:.2f}<br>"
        f"{role_y}: {row['RoleScore_Y']:.2f}<br>"
        f"Minutes: {int(row.get('Mins',0)):,}"
        for _, row in df.iterrows()
    ]
    name_labels = [
        row['Player'] if idx in outlier_indices else ""
        for idx, row in df.iterrows()
    ]

    # 5. Interactive scatter plot (with search)
    base_marker = dict(
        size=9,
        color=df['Age'],
        colorscale="Blues",
        opacity=1,
        line=dict(width=1, color="#333")
    )
    fig = go.FigureWidget([
        go.Scatter(
            x=df['RoleScore_X'],
            y=df['RoleScore_Y'],
            mode='markers+text',
            textposition='top center',
            textfont=dict(color="#222222", size=16, family=FONT_FAMILY),
            text=name_labels,
            marker=base_marker,
            hovertext=hover_texts,
            hoverinfo='text'
        )
    ])

    pos_label = ", ".join(selected_positions) if selected_positions else "All positions"
    fig.update_layout(
            xaxis=dict(
                title=dict(
                    text=role_x,
                    font=dict(family=FONT_FAMILY, size=20, color='#333')
                ),
                showgrid=True,
                gridcolor='#f1ffcd',
                tickfont=dict(family=FONT_FAMILY, size=18, color="#222"),
                zeroline=False,
                zerolinewidth=2,
                zerolinecolor='#000',
                linecolor='#000',
                linewidth=2,
                mirror=True
            ),
            yaxis=dict(
                title=dict(
                    text=role_y,
                    font=dict(family=FONT_FAMILY, size=20, color='#333')
                ),
                showgrid=True,
                gridcolor='#f1ffcd',
                tickfont=dict(family=FONT_FAMILY, size=16, color="#222"),
                zeroline=False,
                zerolinewidth=2,
                zerolinecolor='#000',
                linecolor='#000',
                linewidth=2,
                mirror=True
            ),
                plot_bgcolor='#f1ffcd',
                paper_bgcolor='#f1ffcd',
                height=1400,
                width=1960,
                hoverlabel=dict(bgcolor='white', font_size=13, font_family=FONT_FAMILY)
            )
    

    # Name search widget
    name_search = widgets.Text(
        value='', placeholder='Type player name…', description='Highlight:', layout=widgets.Layout(width='350px')
    )
    def update_highlight(change):
        search_val = name_search.value.strip().lower()
        if not search_val or search_val == "reset":
            opacities = np.ones(len(df))
            sizes = np.full(len(df), 9)
            border_colors = np.full(len(df), "#333")
            updated_names = [
                row['Player'] if idx in outlier_indices else ""
                for idx, row in df.iterrows()
            ]
        else:
            highlight_mask = df['Player'].str.lower().str.contains(search_val)
            opacities = np.where(highlight_mask, 1, 0.23)
            sizes = np.where(highlight_mask, 22, 9)
            border_colors = np.where(highlight_mask, "#B80019", "#A0A0A0")
            updated_names = [
                row['Player'] if highlight_mask[idx] or idx in outlier_indices else ""
                for idx, row in df.iterrows()
            ]
        with fig.batch_update():
            fig.data[0].marker.opacity = opacities
            fig.data[0].marker.size = sizes
            fig.data[0].marker.line.color = border_colors
            fig.data[0].text = updated_names

    name_search.observe(update_highlight, names='value')
    display(widgets.HBox([name_search]))
    ipy_display(fig)
    update_highlight(None)



# ==== MODE HANDLING ====
if mode == "1":
    show_player_summary(player_row, stat_cols, stat_display_names, df)
    show_similarity_comparison(player_row, stat_cols, stat_display_names, df, min_minutes)
elif mode == "2":
    show_percentile_bar_chart(player_row, stat_cols, stat_labels, df, min_minutes)
elif mode == "3":
    show_pizza_chart(player_row, stat_cols, stat_labels, df, min_minutes)
    show_player_summary(player_row, stat_cols, stat_display_names, df)
elif mode == "4":
    show_role_leaderboard(df, position_weights, stat_display_names, min_minutes)
elif mode == "5":
    analyze_player_roles(df, position_weights, stat_display_names, player_row, selected_positions, min_minutes)
elif mode == "6":
    show_top_players_by_stat(df, stat_display_names, min_minutes, selected_positions)
elif mode == "7":
    show_radar_chart(player_row, stat_cols, stat_labels, df, min_minutes)
    print("Radar plots generated for each pizza stat category.")
elif mode == "8":
    pizza_no_toppings(player_row, stat_cols, stat_labels, df, min_minutes)
elif mode == "9":
    show_pizza_chart_lightmode(player_row, stat_cols, stat_labels, df, min_minutes)
elif mode == "10":
    pizza_no_toppings_lightmode(player_row, stat_cols, stat_labels, df, min_minutes)
elif mode == "11":
    calculate_player_synergy(player_row, df, archetype_params, stat_display_names, min_minutes)
elif mode == "12":
    show_stat_scatter(df, stat_display_names, selected_positions)
elif mode == "13":
    show_role_suitability_scatter(df, archetype_params_full, stat_display_names, selected_positions)
else:
    print("Invalid mode selected")



Choose mode:
1. Similar player comparison (bar chart + radar + clipboard)
2. View percentiles for player (horizontal bar chart only)
3. Pizza chart for player
4. Top players by role (graphic leaderboard)
5. Analyze player's best roles
6. View Stat Leaders
7. Generate radar plots for pizza categories
8. Pizza Without The Toppings (For Dashboards)
9. Lightmode Pizza Chart
10. Lightmode Pizza Chart no toppings
11. Ideal Partner Finder
12. Scatter Plot generator
13. Role Suitability Matrix


Enter 1-13:  13



Which DB do you want to load?
1. BigDB_ALL.csv (Minor Leagues)
2. BigDB.csv (Big 5 European Leagues)
3. Both


Enter 1, 2, or 3:  3


Loading: C:\Users\willi\Data Viz\FBREF scrapes\BigDB_ALL.csv
Loading: C:\Users\willi\Data Viz\FBREF scrapes\BigDB.csv
Loaded 9466 total rows from 2 file(s).

Available positions: GK, DF, MF, FW


...


Enter minimum minutes played (default 900):  


Rows after string cleanup: 3192
Rows after numeric coercion: 3192
Rows with 900+ mins: 1256
Final rows after filter: 1256

Archetypes:
 1. CB - Ball-Playing
 2. CB - Stopper
 3. CB - Sweeper
 4. FB/WB - Overlapping Attacker
 5. FB/WB - Inverted
 6. FB - Defensive
 7. DM - Ball Winner
 8. DM - Deep-Lying Playmaker
 9. CM - Box-to-Box Engine
10. CM - Shuttler/Link Player
11. CM - Mezzala
12. AM - Classic 10
13. ST - Target Man
14. ST - Poacher
15. ST - Complete Forward
16. Winger - Classic
17. Winger - Inverted
18. AM - Shadow Striker
19. Passing Pizza
20. Defending Pizza
21. Carrying Pizza
22. Shooting Pizza
23. Aerial Pizza
24. Ball Retention Pizza



Choose archetype for X-axis by number:  17
Choose archetype for Y-axis by number:  13


Selected roles: X-axis = Winger - Inverted, Y-axis = ST - Target Man


FigureWidget({
    'data': [{'hoverinfo': 'text',
              'hovertext': [<b>Thelo Aasgaard</b><br>Team: Luton Town<br>Age:
                            22.0<br>Winger - Inverted: 37.60<br>ST - Target Man:
                            48.00<br>Minutes: 1,513, <b>Elijah Adebayo</b><br>Team:
                            Luton Town<br>Age: 26.0<br>Winger - Inverted:
                            48.00<br>ST - Target Man: 48.10<br>Minutes: 2,425,
                            <b>Lucas Andersen</b><br>Team: Qpr<br>Age:
                            29.0<br>Winger - Inverted: 41.10<br>ST - Target Man:
                            43.20<br>Minutes: 1,165, ..., <b>Zakaria
                            Aboukhlal</b><br>Team: Toulouse<br>Age: 24.0<br>Winger
                            - Inverted: 59.10<br>ST - Target Man: 56.60<br>Minutes:
                            1,901, <b>Zito Luvumbo</b><br>Team: Cagliari<br>Age:
                            22.0<br>Winger - Inverted: 38.80<br>ST - Target Man:
    

In [ ]:
display(df)